In [8]:
# Import des différents packages.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from time import time
import copy
import pickle
from joblib import dump, load

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold

from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

#from imblearn.over_sampling import RandomOverSampler, SMOTE
#from imblearn.under_sampling import RandomUnderSampler,  ClusterCentroids

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
#from imblearn.metrics import classification_report_imbalanced, geometric_mean_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression

In [4]:
from IPython.display import display_markdown
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

In [5]:
import os
root_folder_path = os.path.abspath('..')
data_processed_path = os.path.join(root_folder_path, "data", "processed")

In [49]:
df = pd.read_csv(os.path.join(data_processed_path, r"base_donnees_nettoyee_non_dupliquee.csv"), sep=",", index_col=0)

C:\Users\courio\AppData\Local\Temp\ipykernel_2852\1611470788.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(data_processed_path, r"base_donnees_nettoyee_non_dupliquee.csv"), sep=",", index_col=0)


In [38]:
df.head()

,DateOfCall,CalYear,HourOfCall,IncidentGroup,StopCodeDescription,PropertyCategory,AddressQualifier,Postcode_district,IncGeo_BoroughName,Latitude,...,PlusCode_Description,DateOfCall_Dayofweek,DateOfCall_Quarter,DateOfCall_Month,DateOfCall_Dayofyear,DateOfCall_Dayofmonth,DateOfCall_Weekofyear,DateOfCall_In_Ceconds,MinutesOfCall,FirstPumpArriving_AttendanceTime_InMinutes
0,2009-01-01,2009,0,Special Service,Special Service,Road Vehicle,In street close to gazetteer location,SW11,WANDSWORTH,51.475812,...,Initial Mobilisation,3,1,1,1,1,1,1230768000000000000,0,5.316667
1,2009-01-01,2009,0,Fire,Secondary Fire,Outdoor,In street outside gazetteer location,N9,ENFIELD,51.633342,...,Initial Mobilisation,3,1,1,1,1,1,1230768000000000000,3,5.133333
2,2009-01-01,2009,0,Fire,Secondary Fire,Outdoor,On land associated with building,UB10,HILLINGDON,51.533882,...,Initial Mobilisation,3,1,1,1,1,1,1230768000000000000,4,3.500000
3,2009-01-01,2009,0,Fire,Secondary Fire,Outdoor,In street outside gazetteer location,N7,ISLINGTON,51.551441,...,Initial Mobilisation,3,1,1,1,1,1,1230768000000000000,5,3.883333
4,2009-01-01,2009,0,False Alarm,AFA,Dwelling,Correct incident location,NW5,CAMDEN,51.551297,...,Initial Mobilisation,3,1,1,1,1,1,1230768000000000000,6,2.866667


In [39]:
# categorical_features = list(df.select_dtypes(exclude=[np.number]).columns)
categorical_features = ['DateOfCall',
 'CalYear',
 'HourOfCall',
 'IncidentGroup',
 'StopCodeDescription',
 'PropertyCategory',
 'AddressQualifier',
 'Postcode_district',
 'IncGeo_BoroughName',
 'NumStationsWithPumpsAttending',
 'NumPumpsAttending',
 'PumpCount',
 'NumCalls',
 'Resource_Code',
 'PerformanceReporting',
 'DateAndTimeMobilised',
 'DateAndTimeArrived',
 'DeployedFromStation_Name',
 'DeployedFromLocation',
 'PumpOrder',
 'PlusCode_Code',
 'PlusCode_Description',
 'DateOfCall_Dayofweek',
 'DateOfCall_Quarter',
 'DateOfCall_Month',
 'DateOfCall_Dayofyear',
 'DateOfCall_Dayofmonth',
 'DateOfCall_Weekofyear',
 'MinutesOfCall']

In [40]:
#numeric_features = list(df.select_dtypes(include=[np.number]).columns)
# ,'FirstPumpArriving_AttendanceTime_InMinutes'
numeric_features = ['Latitude','Longitude','PumpMinutesRounded','DateOfCall_In_Ceconds']

In [50]:
y = df['FirstPumpArriving_AttendanceTime_InMinutes']
X = df.drop(['FirstPumpArriving_AttendanceTime_InMinutes'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [51]:
def build_pipeline(algo_ml, impute_num = SimpleImputer(strategy="median"), impute_var=SimpleImputer(strategy="constant", fill_value="Unknown"), scaler = StandardScaler()):
  # Pipeline de transformation des variables numériques
  numeric_transformer = make_pipeline(
    (impute_num), 
    (scaler)
    )
  
  # Pipeline de transformation des variables catégorielles
  categorical_transformer = make_pipeline(
    (impute_var),
    (OneHotEncoder(handle_unknown="ignore"))
    )
  
  # Combinaison des 2 étapes en un seul objet
  preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
    )
  
  # Pipeline final de Machine Learning
  model = Pipeline(steps=[('preprocessing', preprocessor), 
                        ('linear', algo_ml)])
  
  return model

In [56]:
build_pipeline(algo_ml=LinearRegression())

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['Latitude', 'Longitude',
                                                   'PumpMinutesRounded',
                                                   'DateOfCall_In_Ceconds',
                                                   'FirstPumpArriving_AttendanceTime_InMinutes']),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fil...
                                                   'PerformanceReporting',
                                                   'DateAndTimeMobilised',
                                                   'DateAndTimeArrived',
                                                   'DeployedFromStation_Name',
                                                   'DeployedFromLocation',
                                                   'PumpOrder', 'PlusCode_Code',
                                                   'PlusCode_Description',
                                                   'DateOfCall_Dayofweek',
                                                   'DateOfCall_Quarter',
                                                   'DateOfCall_Month',
                                                   'DateOfCall_Dayofyear',
                                                   'DateOfCall_Dayofmonth',
                                                   'DateOfCall_Weekofyear',
                                                   'MinutesOfCall'])])),
                ('linear', LinearRegression())])

In [55]:
model = build_pipeline(algo_ml = LinearRegression())
model

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['Latitude', 'Longitude',
                                                   'PumpMinutesRounded',
                                                   'DateOfCall_In_Ceconds',
                                                   'FirstPumpArriving_AttendanceTime_InMinutes']),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fil...
                                                   'PerformanceReporting',
                                                   'DateAndTimeMobilised',
                                                   'DateAndTimeArrived',
                                                   'DeployedFromStation_Name',
                                                   'DeployedFromLocation',
                                                   'PumpOrder', 'PlusCode_Code',
                                                   'PlusCode_Description',
                                                   'DateOfCall_Dayofweek',
                                                   'DateOfCall_Quarter',
                                                   'DateOfCall_Month',
                                                   'DateOfCall_Dayofyear',
                                                   'DateOfCall_Dayofmonth',
                                                   'DateOfCall_Weekofyear',
                                                   'MinutesOfCall'])])),
                ('linear', LinearRegression())])

In [54]:
LinearRegression().fit(X_train,y_train)

ValueError: could not convert string to float: '2017-01-06'

In [57]:
model.fit(X_train,y_train)

ValueError: A given column is not a column of the dataframe